To what extent did the form predictor that was present in Get Athletes Complex correspond with the actual results? Thinking in terms of ML, what are the variables that can be tweaked and what are the weights to be given to each variable so that we can arrive at a better prediction, now that we have truth-y values?

In [13]:
import pandas as pd
import numpy as np

In [14]:
df = pd.read_csv("sorted_df_exponential.csv")
df

,Mark,Competitor,Nat_x,Location,Discipline,Date,Results Score,Nat_y,Birthdate,Event,Days Since Performance,Weighted Score
0,3:26.73,JAKOB INGEBRIGTSEN,NOR,"Stade Louis II, Monaco (MON)",1500m,2024-07-12,1292,NOR,19 Sep 2000,"1500m, 5000m",20,1266.416686
1,1:41.46,DJAMEL SEDJATI,ALG,"Stade Louis II, Monaco (MON)",800m,2024-07-12,1284,ALG,03 May 1999,800m,20,1258.575097
2,1:41.58,EMMANUEL WANYONYI,KEN,"Stade Charléty, Paris (FRA)",800m,2024-07-07,1280,KEN,01 Aug 2004,800m,25,1248.396687
3,1:42.04,MOHAMED ATTAOUI,ESP,"Stade Louis II, Monaco (MON)",800m,2024-07-12,1265,ESP,26 Sep 2001,800m,20,1239.951322
4,1:42.10,GABRIEL TUAL,FRA,"Stade Louis II, Monaco (MON)",800m,2024-07-12,1264,FRA,09 Apr 1998,800m,20,1238.971123
...,...,...,...,...,...,...,...,...,...,...,...,...
416,1:02:10,OTHMANE EL GOUMRI,MAR,Meishan (CHN),Half Marathon,2024-02-25,1101,MAR,28 May 1992,Marathon,158,940.088610
417,1:01:02,RORY LINKLETTER,CAN,"Houston, TX (USA)",Half Marathon,2024-01-14,1145,CAN,12 Aug 1996,Marathon,200,937.446712
418,13:30.67,JAMES CORRIGAN,USA,"Dempsey Indoor, Seattle, WA (USA)",5000m,2024-02-09,1100,USA,16 Apr 2002,3000m SC,174,924.326587
419,1:47.11,CAMERON MYERS,AUS,"AIS Athletics Track, Canberra (AUS)",800m,2024-01-26,1110,AUS,09 Jun 2006,1500m,188,919.762325


This is based off experiment 1's exponential decay loss to be the graph that fits. For fun, we are going to see how accurate these are per event.

Eventually I think we could do with a simple ML model, but not yet. 

It's a little tricky for distance events because there may be instances when a time ran in the heats / semi-finals rounds are faster than the finals (although unlikely). And which timing to take is also a bit of a bummer. Initial thought is to take the fastest time of the competitor but that does not necessarily match the rankings.

For simplicity's sake, we are going to focus on the final results rather than trying to do a ranking of like all the athletes who qualified.

# 1500m post-race analysis

In [15]:
#what is the most efficient way to collect full results for any race run?

In [16]:
filtered_1500m_df = df[df['Event'].str.contains('1500m')]
filtered_1500m_df

,Mark,Competitor,Nat_x,Location,Discipline,Date,Results Score,Nat_y,Birthdate,Event,Days Since Performance,Weighted Score
0,3:26.73,JAKOB INGEBRIGTSEN,NOR,"Stade Louis II, Monaco (MON)",1500m,2024-07-12,1292,NOR,19 Sep 2000,"1500m, 5000m",20,1266.416686
5,3:28.71,TIMOTHY CHERUIYOT,KEN,"Stade Louis II, Monaco (MON)",1500m,2024-07-12,1263,KEN,20 Nov 1995,1500m,20,1237.990924
6,3:28.80,BRIAN KOMEN,KEN,"Stade Louis II, Monaco (MON)",1500m,2024-07-12,1262,KEN,10 Aug 1998,1500m,20,1237.010726
9,3:29.13,YARED NUGUSE,USA,"Stade Louis II, Monaco (MON)",1500m,2024-07-12,1257,USA,01 Jun 1999,1500m,20,1232.109732
16,3:30.65,NEIL GOURLEY,GBR,"Stade Louis II, Monaco (MON)",1500m,2024-07-12,1235,GBR,07 Feb 1995,1500m,20,1210.545362
...,...,...,...,...,...,...,...,...,...,...,...,...
388,1:47.19,FEDERICO RIVA,ITA,"Stadio Olindo Galli, Tivoli (ITA)",800m,2024-04-25,1108,ITA,09 Nov 2000,1500m,98,1004.566985
406,3:53.16,SAMUEL TANNER,NZL,"Lakeside Stadium, Melbourne (AUS)",Mile,2024-02-15,1160,NZL,24 Aug 2000,1500m,168,980.610448
410,1:46.14,SAMUEL TANNER,NZL,"Nga Puna Wai Sports Hub, Christchurch (NZL)",800m,2024-02-24,1139,NZL,24 Aug 2000,1500m,159,971.562853
414,4:00.71,KIERAN LUMB,CAN,"Hayward Field, Eugene, OR (USA)",Mile,2024-04-20,1066,CAN,02 Aug 1998,1500m,103,961.667354


In [17]:
from bs4 import BeautifulSoup
import requests

In [18]:
def get_olympic_event_results(url, discipline):
    results = []

    # Fetch the webpage
    response = requests.get(url)
    data = response.text
    soup = BeautifulSoup(data, 'html.parser')

    # Locate the table containing the results
    table = soup.find('table')

    # Ensure the table exists
    if table:
        # Iterate over each row in the table (skipping the header row)
        for row in table.find_all('tr')[1:]:
            cols = row.find_all('td')
            if len(cols) > 1:
                place = cols[0].text.strip()
                nationality = cols[2].text.strip()
                competitor = cols[3].text.strip()
                mark = cols[4].text.strip()


                results.append({
                    'Place': place,
                    'Competitor': competitor,
                    'Nat': nationality,
                    'Mark': mark,
                }) #what about the results score? would probably have to scrape data (again) for post OLY results lmao

    # Convert the results into a DataFrame
    df = pd.DataFrame(results)

    # Insert the discipline column
    df.insert(4, 'Discipline', discipline)

    return df


In [19]:
url = "https://worldathletics.org/competitions/olympic-games/paris24/results/men/1500-metres/final/result"
discipline = "1500m"
df_1500_results = get_olympic_event_results(url, discipline)

df_1500_results

,Place,Competitor,Nat,Mark,Discipline
0,1,Cole HOCKER,USA,3:27.65 OR,1500m
1,2,Josh KERR,GBR,3:27.79 NR,1500m
2,3,Yared NUGUSE,USA,3:27.80 PB,1500m
3,4,Jakob INGEBRIGTSEN,NOR,3:28.24,1500m
4,5,Hobbs KESSLER,USA,3:29.45 PB,1500m
5,6,Niels LAROS,NED,3:29.54 NR,1500m
6,7,Narve Gilje NORDÅS,NOR,3:30.46 SB,1500m
7,8,Pietro ARESE,ITA,3:30.74 NR,1500m
8,9,Stefan NILLESSEN,NED,3:30.75 PB,1500m
9,10,Neil GOURLEY,GBR,3:30.88,1500m


In [20]:
df_1500_results['Competitor'] = df_1500_results['Competitor'].str.upper()
df_1500_results


,Place,Competitor,Nat,Mark,Discipline
0,1,COLE HOCKER,USA,3:27.65 OR,1500m
1,2,JOSH KERR,GBR,3:27.79 NR,1500m
2,3,YARED NUGUSE,USA,3:27.80 PB,1500m
3,4,JAKOB INGEBRIGTSEN,NOR,3:28.24,1500m
4,5,HOBBS KESSLER,USA,3:29.45 PB,1500m
5,6,NIELS LAROS,NED,3:29.54 NR,1500m
6,7,NARVE GILJE NORDÅS,NOR,3:30.46 SB,1500m
7,8,PIETRO ARESE,ITA,3:30.74 NR,1500m
8,9,STEFAN NILLESSEN,NED,3:30.75 PB,1500m
9,10,NEIL GOURLEY,GBR,3:30.88,1500m


In [21]:
def find_index_in_larger_df(competitor_name, larger_df):
    try:
        return (larger_df[larger_df['Competitor'] == competitor_name].index[0])+1  # Get the first occurrence
    except IndexError:
        return None  # Return None if the competitor is not found



In [22]:
# Apply the function to the df_1500_results DataFrame
df_1500_results['Index in rerank'] = df_1500_results['Competitor'].apply(find_index_in_larger_df, args=(filtered_1500m_df,))
df_1500_results

,Place,Competitor,Nat,Mark,Discipline,Index in rerank
0,1,COLE HOCKER,USA,3:27.65 OR,1500m,34
1,2,JOSH KERR,GBR,3:27.79 NR,1500m,46
2,3,YARED NUGUSE,USA,3:27.80 PB,1500m,10
3,4,JAKOB INGEBRIGTSEN,NOR,3:28.24,1500m,1
4,5,HOBBS KESSLER,USA,3:29.45 PB,1500m,47
5,6,NIELS LAROS,NED,3:29.54 NR,1500m,32
6,7,NARVE GILJE NORDÅS,NOR,3:30.46 SB,1500m,22
7,8,PIETRO ARESE,ITA,3:30.74 NR,1500m,92
8,9,STEFAN NILLESSEN,NED,3:30.75 PB,1500m,139
9,10,NEIL GOURLEY,GBR,3:30.88,1500m,17


We should compare this rerank (an initial sparse model) with the actual IAAF rankings and see how close / far this is from that. So let's grab the actual IAAF rankings of these competitors, prior to the Olympics. So the earliest date of ranking BEFORE the Olympics is 23 Jul 24, as compared to the latest one that was published AFTER the Olympics (13 Aug 24).

## Selenium Config

This is selenium config especially since the rankings are most probably dynamically rendered.

In [1]:
!pip install selenium

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 51.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 476.0/476.0 kB 31.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.6 MB/s eta 0:00:00


In [2]:
!apt-get update
!apt-get install chromium chromium-driver

Hit:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:2 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Ign:3 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 https://r2u.stat.illinois.edu/ubuntu jammy Release [5,713 B]
Get:6 https://r2u.stat.illinois.edu/ubuntu jammy Release.gpg [793 B]
Get:7 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:8 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:9 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 Packages [2,553 kB]
Get:10 http://security.ubuntu.com/ubuntu jammy-security/restricted amd64 Packages [2,867 kB]
Get:11 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:12 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages [1,422 kB]
Get:13 http://archive.ubuntu.com/ubuntu jammy-updates/restricted amd64 Packages [2,963 kB]

In [3]:
from selenium import webdriver

In [4]:
!apt-get install chromium-driver

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
Note, selecting 'chromium-chromedriver' instead of 'chromium-driver'
The following additional packages will be installed:
  apparmor chromium-browser libfuse3-3 liblzo2-2 libudev1 snapd squashfs-tools systemd-hwe-hwdb
  udev
Suggested packages:
  apparmor-profiles-extra apparmor-utils fuse3 zenity | kdialog
The following NEW packages will be installed:
  apparmor chromium-browser chromium-chromedriver libfuse3-3 liblzo2-2 snapd squashfs-tools
  systemd-hwe-hwdb udev
The following packages will be upgraded:
  libudev1
1 upgraded, 9 newly installed, 0 to remove and 44 not upgraded.
Need to get 28.5 MB of archives.
After this operation, 118 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 apparmor amd64 3.0.4-2ubuntu2.3 [595 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/main amd64 liblzo2-2 amd64 2.10-2build3 [53.7 kB]
Get:3 http://ar

In [5]:
def web_driver():
    options = webdriver.ChromeOptions()
    options.add_argument("--verbose")
    options.add_argument('--no-sandbox')
    options.add_argument('--headless')
    options.add_argument('--disable-gpu')
    options.add_argument("--window-size=1920, 1200")
    options.add_argument('--disable-dev-shm-usage')
    driver = webdriver.Chrome(options=options)
    return driver



In [6]:
import pandas as pd
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import time
from bs4 import BeautifulSoup

In [7]:
driver = web_driver()

# Fetching 1500m athlete data

In [9]:
def fetch_athlete_data(driver, urls):
    all_athletes = []

    for url in urls:
        driver.get(url)

        # Wait for the table to be present on the page
        try:
            WebDriverWait(driver, 20).until(
                EC.presence_of_element_located((By.CLASS_NAME, "records-table"))
            )
        except Exception as e:
            print(f"An error occurred while waiting for the table at {url}:", e)
            continue


        time.sleep(5)

        raw_html = driver.page_source

        soup = BeautifulSoup(raw_html, 'html.parser')


        table = soup.find('table', class_='records-table')
        if table:
            for row in table.find_all('tr')[1:]:
                cols = row.find_all('td')
                if len(cols) > 1:
                    rank = cols[0].text.strip()
                    athlete = cols[1].text.strip()
                    nationality = cols[2].text.strip()
                    score = cols[3].text.strip()
                    event_list = cols[4].text.strip()

                    all_athletes.append({
                        'Rank': rank,
                        'Athlete': athlete,
                        'Nat': nationality,
                        'Score': score,
                        'Event List': event_list
                    })
        else:
            print(f"Table not found at {url}")

    # Convert to DataFrame
    df = pd.DataFrame(all_athletes)
    return df


# Example usage:
# Assuming driver is a Selenium WebDriver instance that has been properly configured
urls = [
    "https://worldathletics.org/world-rankings/1500m/men?regionType=world&page=1&rankDate=2024-07-23&limitByCountry=0",  # Replace with actual URL
    "https://worldathletics.org/world-rankings/1500m/men?regionType=world&page=2&rankDate=2024-07-23&limitByCountry=0"   # Replace with actual URL
]




In [11]:
# Fetch data using the driver and the provided URLs and events
df_rankings = fetch_athlete_data(driver, urls)

df_rankings

,Rank,Athlete,Nat,Score,Event List
0,1,Jakob INGEBRIGTSEN,19 SEP 2000,NOR,1520
1,2,Yared NUGUSE,01 JUN 1999,USA,1449
2,3,Josh KERR,08 OCT 1997,GBR,1391
3,4,Narve Gilje NORDÅS,30 SEP 1998,NOR,1390
4,5,Reynold CHERUIYOT,30 JUL 2004,KEN,1389
...,...,...,...,...,...
195,196,Anas LAGITY CHAOUDAR,17 JAN 2004,FRA,1126
196,197,Sigurd TVEIT,16 MAR 2000,NOR,1126
197,198,Thiago ANDRÉ,04 AUG 1995,BRA,1125
198,199,Tomas PALFREY,08 FEB 2003,AUS,1125


In [12]:

df_1500_rankings = df_rankings.rename(columns={'Athlete': 'Competitor'})
df_1500_rankings['Competitor'] = df_1500_rankings['Competitor'].str.upper()
df_1500_rankings


,Rank,Competitor,Nat,Score,Event List
0,1,JAKOB INGEBRIGTSEN,19 SEP 2000,NOR,1520
1,2,YARED NUGUSE,01 JUN 1999,USA,1449
2,3,JOSH KERR,08 OCT 1997,GBR,1391
3,4,NARVE GILJE NORDÅS,30 SEP 1998,NOR,1390
4,5,REYNOLD CHERUIYOT,30 JUL 2004,KEN,1389
...,...,...,...,...,...
195,196,ANAS LAGITY CHAOUDAR,17 JAN 2004,FRA,1126
196,197,SIGURD TVEIT,16 MAR 2000,NOR,1126
197,198,THIAGO ANDRÉ,04 AUG 1995,BRA,1125
198,199,TOMAS PALFREY,08 FEB 2003,AUS,1125


In [24]:
# Apply the function to the df_1500_results DataFrame
df_1500_results['Index in IAAF rank'] = df_1500_results['Competitor'].apply(find_index_in_larger_df, args=(df_1500_rankings,))
df_1500_results

,Place,Competitor,Nat,Mark,Discipline,Index in rerank,Index in IAAF rank
0,1,COLE HOCKER,USA,3:27.65 OR,1500m,34,8
1,2,JOSH KERR,GBR,3:27.79 NR,1500m,46,3
2,3,YARED NUGUSE,USA,3:27.80 PB,1500m,10,2
3,4,JAKOB INGEBRIGTSEN,NOR,3:28.24,1500m,1,1
4,5,HOBBS KESSLER,USA,3:29.45 PB,1500m,47,12
5,6,NIELS LAROS,NED,3:29.54 NR,1500m,32,13
6,7,NARVE GILJE NORDÅS,NOR,3:30.46 SB,1500m,22,4
7,8,PIETRO ARESE,ITA,3:30.74 NR,1500m,92,26
8,9,STEFAN NILLESSEN,NED,3:30.75 PB,1500m,139,48
9,10,NEIL GOURLEY,GBR,3:30.88,1500m,17,16


This is interesting. I think this gives me some baseline reranking to start with! Loss could be something like difference in ranked index vs actual placing.

In [57]:

import pandas as pd
# Convert index columns to numeric, handling non-numeric values
df_1500_results['Index in rerank'] = pd.to_numeric(df_1500_results['Index in rerank'], errors='coerce')
df_1500_results['Index in IAAF rank'] = pd.to_numeric(df_1500_results['Index in IAAF rank'], errors='coerce')
df_1500_results['Place'] = pd.to_numeric(df_1500_results['Place'], errors='coerce')

# Calculate differences and squared errors
df_1500_results['Diff from Rerank'] = (df_1500_results['Index in rerank'] - df_1500_results['Place']).fillna(0)
df_1500_results['Diff from IAAF'] = (df_1500_results['Index in IAAF rank'] - df_1500_results['Place']).fillna(0)

df_1500_results['Squared Error Rerank'] = df_1500_results['Diff from Rerank'] ** 2
df_1500_results['Squared Error IAAF'] = df_1500_results['Diff from IAAF'] ** 2

# Calculate mean squared errors
rmse_rerank = np.sqrt(df_1500_results['Squared Error Rerank'].mean())
rmse_iaaf = np.sqrt(df_1500_results['Squared Error IAAF'].mean())

print("Mean Squared Error (Rerank):", rmse_rerank)
print("Mean Squared Error (IAAF):", rmse_iaaf)

df_1500_results


Mean Squared Error (Rerank): 49.83556293785928
Mean Squared Error (IAAF): 13.231905884389192


,Place,Competitor,Nat,Mark,Discipline,Index in rerank,Index in IAAF rank,Diff from Rerank,Diff from IAAF,Squared Error Rerank,Squared Error IAAF
0,1,COLE HOCKER,USA,3:27.65 OR,1500m,34,8,33,7,1089,49
1,2,JOSH KERR,GBR,3:27.79 NR,1500m,46,3,44,1,1936,1
2,3,YARED NUGUSE,USA,3:27.80 PB,1500m,10,2,7,-1,49,1
3,4,JAKOB INGEBRIGTSEN,NOR,3:28.24,1500m,1,1,-3,-3,9,9
4,5,HOBBS KESSLER,USA,3:29.45 PB,1500m,47,12,42,7,1764,49
5,6,NIELS LAROS,NED,3:29.54 NR,1500m,32,13,26,7,676,49
6,7,NARVE GILJE NORDÅS,NOR,3:30.46 SB,1500m,22,4,15,-3,225,9
7,8,PIETRO ARESE,ITA,3:30.74 NR,1500m,92,26,84,18,7056,324
8,9,STEFAN NILLESSEN,NED,3:30.75 PB,1500m,139,48,130,39,16900,1521
9,10,NEIL GOURLEY,GBR,3:30.88,1500m,17,16,7,6,49,36


So in this case, only my model of exponential loss decay predicted that Timothy Cheruiyot raced better than his actual placing. interesting!

In [26]:
df_1500_results.to_csv("1500m_results_against_exponential.csv")

So the model doesn't do great, with 1 prediction better than compared to IAAF rankings. How does it fair against the rest of the distances?

# Rest of endurance events analysis

So now, the flow is pretty straightforward:
1. Grab results from worldathletics
2. Filter all 3000m SC entries in exponential loss decay fn df
3. Grab index from worldathletics ranking
4. Compare mean squared error of placing vs actual

In [32]:
def get_filtered_event_df(df, event):
    return df[df['Event'].str.contains(event)]


In [51]:
def complete_analysis(result_url, event, ranking_urls):
    df_results = get_olympic_event_results(result_url, event)
    filtered_event_df = get_filtered_event_df(df, event)
    df_results['Competitor'] = df_results['Competitor'].str.upper()
    df_results['Index in rerank'] = df_results['Competitor'].apply(find_index_in_larger_df, args=(filtered_event_df,)) #okay this doesn't work quite well now
    driver = web_driver()
    df_rankings = fetch_athlete_data(driver, ranking_urls)
    df_rankings = df_rankings.rename(columns={'Athlete': 'Competitor'})
    df_rankings['Competitor'] = df_rankings['Competitor'].str.upper()
    df_results['Index in IAAF rank'] = df_results['Competitor'].apply(find_index_in_larger_df, args=(df_rankings,))
    df_results['Index in rerank'] = pd.to_numeric(df_results['Index in rerank'], errors='coerce')
    df_results['Index in IAAF rank'] = pd.to_numeric(df_results['Index in IAAF rank'], errors='coerce')
    df_results['Place'] = pd.to_numeric(df_results['Place'], errors='coerce')

# Calculate differences and squared errors
    df_results['Diff from Rerank'] = (df_results['Index in rerank'] - df_results['Place']).fillna(0)
    df_results['Diff from IAAF'] = (df_results['Index in IAAF rank'] - df_results['Place']).fillna(0)

    df_results['Mean Squared Error Rerank'] = (df_results['Diff from Rerank'] ** 2)
    df_results['Mean Squared Error IAAF'] = (df_results['Diff from IAAF'] ** 2)

    # Calculate root mean squared errors
    rmse_rerank = np.sqrt(df_results['Mean Squared Error Rerank'].mean())
    rmse_iaaf = np.sqrt(df_results['Mean Squared Error IAAF'].mean())

    print("Root Mean Squared Error (Rerank):", rmse_rerank)
    print("Root Mean Squared Error (IAAF):", rmse_iaaf)


    return df_results


In [52]:
complete_analysis("https://worldathletics.org/competitions/olympic-games/paris24/results/men/3000-metres-steeplechase/final/result", "3000m SC", ["https://worldathletics.org/world-rankings/3000msc/men?regionType=world&page=1&rankDate=2024-07-23&limitByCountry=0", "https://worldathletics.org/world-rankings/3000msc/men?regionType=world&page=2&rankDate=2024-07-23&limitByCountry=0"] )

Root Mean Squared Error (Rerank): 104.02974334294976
Root Mean Squared Error (IAAF): 14.49784466739798


,Place,Competitor,Nat,Mark,Discipline,Index in rerank,Index in IAAF rank,Diff from Rerank,Diff from IAAF,Mean Squared Error Rerank,Mean Squared Error IAAF
0,1.0,SOUFIANE EL BAKKALI,MAR,8:06.05 SB,3000m SC,136,1,135.0,0.0,18225.0,0.0
1,2.0,KENNETH ROOKS,USA,8:06.41 PB,3000m SC,176,24,174.0,22.0,30276.0,484.0
2,3.0,ABRAHAM KIBIWOT,KEN,8:06.47 SB,3000m SC,29,3,26.0,0.0,676.0,0.0
3,4.0,MOHAMED AMIN JHINAOUI,TUN,8:07.73 NR,3000m SC,37,11,33.0,7.0,1089.0,49.0
4,5.0,AHMED JAZIRI,TUN,8:08.02 PB,3000m SC,128,56,123.0,51.0,15129.0,2601.0
5,6.0,SAMUEL FIREWU,ETH,8:08.87,3000m SC,72,4,66.0,-2.0,4356.0,4.0
6,7.0,SIMON KIPROP KOECH,KEN,8:09.26 SB,3000m SC,187,8,180.0,1.0,32400.0,1.0
7,8.0,RYUJI MIURA,JPN,8:11.72,3000m SC,45,9,37.0,1.0,1369.0,1.0
8,9.0,GETNET WALE,ETH,8:12.33,3000m SC,117,7,108.0,-2.0,11664.0,4.0
9,10.0,DANIEL ARCE,ESP,8:13.80,3000m SC,181,14,171.0,4.0,29241.0,16.0


Well that didn't go very well for 3000m SC, let's see how it goes for the rest of the events:

### 800m

In [53]:
complete_analysis("https://worldathletics.org/competitions/olympic-games/paris24/results/men/800-metres/final/result", "800m", ["https://worldathletics.org/world-rankings/800m/men?regionType=world&page=1&rankDate=2024-07-23&limitByCountry=0", "https://worldathletics.org/world-rankings/800m/men?regionType=world&page=2&rankDate=2024-07-23&limitByCountry=0"])

Root Mean Squared Error (Rerank): 52.15361924162119
Root Mean Squared Error (IAAF): 41.11265012134343


,Place,Competitor,Nat,Mark,Discipline,Index in rerank,Index in IAAF rank,Diff from Rerank,Diff from IAAF,Mean Squared Error Rerank,Mean Squared Error IAAF
0,1,EMMANUEL WANYONYI,KEN,1:41.19 PB,800m,3,1,2,0,4,0
1,2,MARCO AROP,CAN,1:41.20 AR,800m,16,2,14,0,196,0
2,3,DJAMEL SEDJATI,ALG,1:41.50,800m,2,3,-1,0,1,0
3,4,BRYCE HOPPEL,USA,1:41.67 NR,800m,24,4,20,0,400,0
4,5,MOHAMED ATTAOUI,ESP,1:42.08,800m,4,13,-1,8,1,64
5,6,GABRIEL TUAL,FRA,1:42.14,800m,5,7,-1,1,1,1
6,7,TSHEPISO MASALELA,BOT,1:42.82 PB,800m,101,8,94,1,8836,1
7,8,MAX BURGIN,GBR,1:43.84,800m,119,124,111,116,12321,13456


Very nice! 800m is a lot better. Perhaps this is better for short distance racing as compared to long distance racing? But then again there are more factors that come into play than long didstance racing. So maybe for things like 400m, 400mH, 800m?

In [54]:
complete_analysis("https://worldathletics.org/competitions/olympic-games/paris24/results/men/5000-metres/final/result", "5000m", ["https://worldathletics.org/world-rankings/5000m/men?regionType=world&page=1&rankDate=2024-07-23&limitByCountry=0", "https://worldathletics.org/world-rankings/5000m/men?regionType=world&page=2&rankDate=2024-07-23&limitByCountry=0"])

Root Mean Squared Error (Rerank): 163.25732616165973
Root Mean Squared Error (IAAF): 29.077482697097423


,Place,Competitor,Nat,Mark,Discipline,Index in rerank,Index in IAAF rank,Diff from Rerank,Diff from IAAF,Mean Squared Error Rerank,Mean Squared Error IAAF
0,1,JAKOB INGEBRIGTSEN,NOR,13:13.66 SB,5000m,1.0,2.0,0.0,1.0,0.0,1.0
1,2,RONALD KWEMOI,KEN,13:15.04,5000m,270.0,45.0,268.0,43.0,71824.0,1849.0
2,3,GRANT FISHER,USA,13:15.13,5000m,112.0,5.0,109.0,2.0,11881.0,4.0
3,4,DOMINIC LOKINYOMO LOBALU,EOR,13:15.27,5000m,267.0,7.0,263.0,3.0,69169.0,9.0
4,5,HAGOS GEBRHIWET,ETH,13:15.32,5000m,11.0,3.0,6.0,-2.0,36.0,4.0
5,6,BINIAM MEHARY,ETH,13:15.99,5000m,25.0,9.0,19.0,3.0,361.0,9.0
6,7,EDWIN KURGAT,KEN,13:17.18,5000m,74.0,NaN,67.0,0.0,4489.0,0.0
7,8,ISAAC KIMELI,BEL,13:18.10,5000m,131.0,37.0,123.0,29.0,15129.0,841.0
8,9,GRAHAM BLANKS,USA,13:18.67,5000m,138.0,61.0,129.0,52.0,16641.0,2704.0
9,10,JACOB KROP,KEN,13:18.68 SB,5000m,310.0,12.0,300.0,2.0,90000.0,4.0


Not entirely sure why Hugo Hay is a NaN and a name that didn't appear in sorted_df_exponential. Was he a last minute replacement? Need to check this

In [55]:
complete_analysis("https://worldathletics.org/en/competitions/olympic-games/paris24/results/men/10000-metres/final/result", "10000m", ["https://worldathletics.org/world-rankings/10000m/men?regionType=world&page=1&rankDate=2024-07-23&limitByCountry=0", "https://worldathletics.org/world-rankings/10000m/men?regionType=world&page=2&rankDate=2024-07-23&limitByCountry=0"])

Root Mean Squared Error (Rerank): 131.43833929655776
Root Mean Squared Error (IAAF): 28.996168329242707


,Place,Competitor,Nat,Mark,Discipline,Index in rerank,Index in IAAF rank,Diff from Rerank,Diff from IAAF,Mean Squared Error Rerank,Mean Squared Error IAAF
0,1.0,JOSHUA CHEPTEGEI,UGA,26:43.14 OR,10000m,68.0,2.0,67.0,1.0,4489.0,1.0
1,2.0,BERIHU AREGAWI,ETH,26:43.44,10000m,14.0,4.0,12.0,2.0,144.0,4.0
2,3.0,GRANT FISHER,USA,26:43.46 SB,10000m,112.0,11.0,109.0,8.0,11881.0,64.0
3,4.0,MOHAMMED AHMED,CAN,26:43.79 SB,10000m,132.0,5.0,128.0,1.0,16384.0,1.0
4,5.0,BENARD KIBET,KEN,26:43.98 PB,10000m,73.0,6.0,68.0,1.0,4624.0,1.0
5,6.0,YOMIF KEJELCHA,ETH,26:44.02,10000m,15.0,7.0,9.0,1.0,81.0,1.0
6,7.0,SELEMON BAREGA,ETH,26:44.48,10000m,19.0,3.0,12.0,-4.0,144.0,16.0
7,8.0,JACOB KIPLIMO,UGA,26:46.39 SB,10000m,26.0,NaN,18.0,0.0,324.0,0.0
8,9.0,THIERRY NDIKUMWENAYO,ESP,26:49.49 NR,10000m,219.0,38.0,210.0,29.0,44100.0,841.0
9,10.0,ADRIAAN WILDSCHUTT,RSA,26:50.64 NR,10000m,98.0,10.0,88.0,0.0,7744.0,0.0


In [56]:
complete_analysis("https://worldathletics.org/en/competitions/olympic-games/paris24/results/men/marathon/final/result", "Marathon", ["https://worldathletics.org/world-rankings/marathon/men?regionType=world&page=1&rankDate=2024-07-23&limitByCountry=0", "https://worldathletics.org/world-rankings/marathon/men?regionType=world&page=2&rankDate=2024-07-23&limitByCountry=0"])

Root Mean Squared Error (Rerank): 245.42829784422995
Root Mean Squared Error (IAAF): 66.1949048802003


,Place,Competitor,Nat,Mark,Discipline,Index in rerank,Index in IAAF rank,Diff from Rerank,Diff from IAAF,Mean Squared Error Rerank,Mean Squared Error IAAF
0,1.0,TAMIRAT TOLA,ETH,2:06:26 OR,Marathon,366.0,5.0,365.0,4.0,133225.0,16.0
1,2.0,BASHIR ABDI,BEL,2:06:47 SB,Marathon,NaN,12.0,0.0,10.0,0.0,100.0
2,3.0,BENSON KIPRUTO,KEN,2:07:00,Marathon,183.0,1.0,180.0,-2.0,32400.0,4.0
3,4.0,EMILE CAIRESS,GBR,2:07:29,Marathon,218.0,39.0,214.0,35.0,45796.0,1225.0
4,5.0,DERESA GELETA,ETH,2:07:31,Marathon,258.0,35.0,253.0,30.0,64009.0,900.0
...,...,...,...,...,...,...,...,...,...,...,...
76,NaN,AMANAL PETROS,GER,DNF,Marathon,208.0,50.0,0.0,0.0,0.0,0.0
77,NaN,STEPHEN KISSA,UGA,DNF,Marathon,347.0,37.0,0.0,0.0,0.0,0.0
78,NaN,KAAN KIGEN ÖZBILEN,TUR,DNF,Marathon,313.0,76.0,0.0,0.0,0.0,0.0
79,NaN,CHRISTIAN PACHECO,PER,DNF,Marathon,NaN,171.0,0.0,0.0,0.0,0.0


Need to check why there are some NaNs in the rerank section!

So comparing the differences in root mean squared error for index in exponential loss decay rerank vs IAAF ranking, there's a general trend that RMSE increases with increasing distance. So probably not the best model for longer distances.

In [59]:
import pandas as pd

data = {
    'Event': ['800m', '1500m', '3000m SC', '5000m', '10000m', 'Marathon'],
    'MSE Exponential Decay Rerank': [52.15361924162119, 49.83556293785928, 104.02974334294976, 163.25732616165973, 131.43833929655776, 245.42829784422995],
    'MSE IAAF Rank': [41.11265012134343, 13.231905884389192, 14.49784466739798, 29.077482697097423, 28.996168329242707, 66.1949048802003]
}

RMSEdf = pd.DataFrame(data)
RMSEdf


,Event,MSE Exponential Decay Rerank,MSE IAAF Rank
0,800m,52.153619,41.112650
1,1500m,49.835563,13.231906
2,3000m SC,104.029743,14.497845
3,5000m,163.257326,29.077483
4,10000m,131.438339,28.996168
5,Marathon,245.428298,66.194905


# Depre

Okay, BS4 for rankings don't work.

In [14]:
url_1500_rank_1 = ["https://worldathletics.org/world-rankings/1500m/men?regionType=world&page=1&rankDate=2024-07-23&limitByCountry=0"]
url_1500_rank_2 = ["https://worldathletics.org/world-rankings/1500m/men?regionType=world&page=2&rankDate=2024-07-23&limitByCountry=0"]
urls = [url_1500_rank_1, url_1500_rank_2]

In [17]:
def get_olympic_IAAF_ranking_results(urls, discipline):
    results = []

    # Loop through each URL in the list
    for url in urls:
        # Fetch the webpage
        response = requests.get(url)
        data = response.text
        soup = BeautifulSoup(data, 'html.parser')

        # Locate the table containing the results
        table = soup.find('table')

        # Ensure the table exists
        if table:
            # Iterate over each row in the table (skipping the header row)
            for row in table.find_all('tr')[1:]:
                cols = row.find_all('td')
                if len(cols) > 1:
                    place = cols[0].text.strip()
                    competitor = cols[1].text.strip()
                    dob = cols[2].text.strip()
                    nationality = cols[3].text.strip()
                    score = cols[4].text.strip()
                    event_list = cols[5].text.strip()

                    results.append({
                        'Place': place,
                        'Competitor': competitor,
                        'DOB': dob,
                        'Nat': nationality,
                        'Score': score,
                        'Event List': event_list
                    })

    # Convert the results into a DataFrame
    df = pd.DataFrame(results)

    # Insert the discipline column
    df.insert(6, 'Discipline', discipline)

    return df

In [18]:
get_olympic_IAAF_ranking_results(urls, "1500m")

InvalidSchema: No connection adapters were found for "['https://worldathletics.org/world-rankings/1500m/men?regionType=world&page=1&rankDate=2024-07-23&limitByCountry=0']"

In [30]:
url = "https://worldathletics.org/competitions/olympic-games/paris24/results/men/3000-metres-steeplechase/final/result"
discipline = "steeple"
df_steeple_results = get_olympic_event_results(url, discipline)

df_steeple_results

,Place,Competitor,Nat,Mark,Discipline
0,1,Soufiane EL BAKKALI,MAR,8:06.05 SB,steeple
1,2,Kenneth ROOKS,USA,8:06.41 PB,steeple
2,3,Abraham KIBIWOT,KEN,8:06.47 SB,steeple
3,4,Mohamed Amin JHINAOUI,TUN,8:07.73 NR,steeple
4,5,Ahmed JAZIRI,TUN,8:08.02 PB,steeple
5,6,Samuel FIREWU,ETH,8:08.87,steeple
6,7,Simon Kiprop KOECH,KEN,8:09.26 SB,steeple
7,8,Ryuji MIURA,JPN,8:11.72,steeple
8,9,Getnet WALE,ETH,8:12.33,steeple
9,10,Daniel ARCE,ESP,8:13.80,steeple
